In [5]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
import sqlite3
from sklearn.metrics import r2_score

# establish sql connection
db_path = "data/input_data.db"
conn = sqlite3.connect(db_path)
cur = conn.cursor()

In [ ]:
# # old
# df_forecast = pd.read_excel('data/Forecast_11_to_14.xlsx', usecols=['date', 'pvpower_instant'])
# df_forecast['date'] = pd.to_datetime(df_forecast['date'], format='%Y-%m-%dT%H:%M+00:00')
# df_forecast = df_forecast.loc[(df_forecast['date'].dt.day == 12) | (df_forecast['date'].dt.day == 13)]
# df_real = pd.read_excel('data/PV Generation-data-2022-11-01 12_52_19_neu.xlsx')
# df_real['Timestamp'] = pd.to_datetime(df_real['Timestamp'], format='%d.%m.%Y %H:%M:%S')

# fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace(go.Scatter(x=df_real['Timestamp'], y=df_real['Heisenberg/71/260_READ_Power_DC1'], name='Real PV-Power'), secondary_y=True)
# fig.add_trace(go.Scatter(x=df_forecast['date'], y=df_forecast['pvpower_instant'], name='Meteoblue Forecast'), secondary_y=False)
# fig.update_layout(title_text='Vergleich MeteoBlue Forecast vs. Messdaten', title_x=0.5, template='plotly',
#                   xaxis_title='Zeit', xaxis=dict(tickformat='%d.%m %H:%M'),
#                   legend=dict(orientation='h', yanchor="top", y=0.99, xanchor="center", x=0.45))
# fig.update_yaxes(title_text='PV-Messwerte', secondary_y=True)
# fig.update_yaxes(title_text='MeteoBlue Forecast', secondary_y=False)
# fig.show()

Import all data from SQL

In [23]:

#timerange = "Timestamp between '2022-07-19 00:00:00' and '2022-07-20 00:00:00' ORDER by Timestamp"
baseline=[pd.read_sql_query("SELECT * FROM baseline", conn).set_index('Timestamp'),'baseline']
household_batteries=[pd.read_sql_query("SELECT * FROM household_batteries ", conn).set_index('Timestamp'), 'household_batteries']
mb_basic=[pd.read_sql_query("SELECT * FROM mb_basic ", conn).set_index('Timestamp'),'mb_basic']
mb_pvpro_1h = [pd.read_sql_query("SELECT * FROM mb_pvpro_1h ", conn).set_index('Timestamp'), 'mb_pvpro_1h']
mb_clouds =[ pd.read_sql_query("SELECT * FROM mb_clouds ", conn).set_index('Timestamp'), 'mb_clouds']
mb_solar = [pd.read_sql_query("SELECT * FROM mb_solar ", conn).set_index('Timestamp'), 'mb_solar']
mb_sunmoon = [pd.read_sql_query("SELECT * FROM mb_sunmoon ", conn).set_index('Timestamp'), 'mb_sunmoon']
slp = [pd.read_sql_query("SELECT * FROM slp ", conn).set_index('Timestamp'), 'slp']
mb_pvpro_15min = [pd.read_sql_query("SELECT * FROM mb_pvpro_15min ", conn).set_index('Timestamp'), 'mb_pvpro_15min']
wunderground_historical_40 = [pd.read_sql_query("SELECT * FROM wunderground_historical_40 ", conn).set_index('Date'), 'wunderground_historical_40' ]
wunderground_historical_43 = [pd.read_sql_query("SELECT * FROM wunderground_historical_43 ", conn).set_index('Date'), 'wunderground_historical_43' ]
wunderground_historical_25 = [pd.read_sql_query("SELECT * FROM wunderground_historical_25 ", conn).set_index('Date'), 'wunderground_historical_25']

# baseline.name='baseline'
# household_batteries.name ='household_batteries'
# mb_basic.name = 'mb_basic'
# mb_pvpro_1h.name = 'mb_pvpro_1h'
# mb_clouds.name = 'mb_clouds'
# mb_solar.name = 'mb_solar'
# mb_sunmoon.name = 'mb_sunmoon'
# slp.name = 'slp'
# mb_pvpro_15min.name = 'mb_pvpro_15min'
# wunderground_historical_40.name = 'wunderground_historical_40' 
# wunderground_historical_43.name = 'wunderground_historical_43' 
# wunderground_historical_25.name = 'wunderground_historical_25'

df_list=[baseline, household_batteries, mb_basic, mb_pvpro_1h, mb_clouds, mb_solar, mb_sunmoon, slp, mb_pvpro_15min, wunderground_historical_40, wunderground_historical_43, wunderground_historical_25]
for df in df_list:
    df[0].index=pd.to_datetime(df[0].index)
    

print all data

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

for df in df_list:
    tempDf = df[0].loc['2022-10-01 00:00:00': '2022-10-02 00:00:00']     #hier Zeitraum auswählen; aber nicht zu groß, sonst schmiert das hier ab
    for c in tempDf.columns:
        if c in ['Baseline in kW' ]:
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=True)
        elif c not in ['rainspot', 'visibility']:  
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=False)

fig.show()
#fig.show(renderer='browser')

Interessante Tage:

    Keine/wenig Verschattung:
        '2022-09-21 00:00:00'
        '2022-10-06 00:00:00'
        
    komplette Verschattung:
        '2022-09-24 00:00:00'
        '2022-04-04 00:00:00'

    teilweise bewölkt:
        '2022-10-17 00:00:00'
        '2022-10-16 00:00:00'
        '2022-10-08 00:00:00'

print selected data

In [26]:
fig = make_subplots(specs=[[{'secondary_y': True}]])

for df in df_list:
    tempDf = df[0].loc['2022-09-15 00:00:00': '2022-11-14 00:00:00']     #hier Zeitraum auswählen von bis
    for c in tempDf.columns:
        if c in ['Baseline in kW','solar_radiation', 'pvpower_instant']:             #hier Reihe für 1. Achse auswählen
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=True)
        elif c in [ 'Pascal', 'Heisenberg', 'Einstein', 'Kelvin', 'Tesla']:                                   #hier Reihe für 2. Achse auswählen
            fig.add_trace(go.Scatter(x=tempDf.index, y=tempDf[c], name=df[1]+' '+c), secondary_y=False)

fig.update_layout(template='plotly')
fig.show(renderer = 'browser')
#pups

C:\Users\bo112\AppData\Local\Temp\ipykernel_6724\2919536881.py:4: FutureWarning:

Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.



PV Vergleich von Max

In [15]:
df_households = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', conn, parse_dates=['Timestamp'])
df_station40 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_40', conn)
df_station43 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_43', conn)
df_station25 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_25', conn)
df_mb_15 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_15min', conn, parse_dates=['Timestamp'])

df_mb_15 = df_mb_15.resample('1Min', on='Timestamp').mean().ffill()
result_df = df_mb_15.merge(df_households, on='Timestamp', how='inner')

In [27]:
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=result_df['Timestamp'], y=result_df[['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla']]), secondary_y=True)
fig.add_trace(go.Scatter(x=result_df['Timestamp'], y=result_df['pvpower_instant']), secondary_y=False)
fig.show(renderer='browser')

R2_score

In [3]:
# read in all PV-related data -> interpolation: 1min timesteps / mean: 15min timesteps
resample_method = 'mean'

df_households = pd.read_sql_query('SELECT Timestamp, Pascal, Einstein, Heisenberg, Kelvin, Tesla FROM household_batteries', conn, parse_dates=['Timestamp'])
df_station40 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_40', conn, parse_dates=['Date'])
df_station40.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_station43 = pd.read_sql_query('SELECT Date, solar_radiation FROM wunderground_historical_43', conn, parse_dates=['Date'])
df_station43.rename(columns={'Date': 'Timestamp'}, inplace=True)
df_mb_15 = pd.read_sql_query('SELECT Timestamp, pvpower_instant FROM mb_pvpro_15min', conn, parse_dates=['Timestamp'])

if resample_method == 'interpolation':
    df_mb_15 = df_mb_15.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station40 = df_station40.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station43 = df_station43.resample('1Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_results = df_households.merge(df_mb_15, on='Timestamp', how='inner')
elif resample_method == 'mean':
    df_households = df_households.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station40 = df_station40.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_station43 = df_station43.resample('15Min', on='Timestamp').mean().ffill(limit=1).interpolate()
    df_results = df_mb_15.merge(df_households, on='Timestamp', how='inner')
else:
    print('WRONG RESAMPLE METHOD!!!')
    raise KeyboardInterrupt

df_results = df_results.merge(df_station40, on='Timestamp', how='inner')
df_results = df_results.merge(df_station43, on='Timestamp', how='inner', suffixes=('_40', '_43'))
df_results['sum_households'] = df_results[['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla']].sum(axis=1)

del df_households, df_station40, df_station43, df_mb_15

In [10]:
# correlate every days measured power


r2_base = 'pvpower_instant'
fig = make_subplots(specs=[[{'secondary_y': True}]])

df_r2 = pd.DataFrame()
cols = ['Pascal', 'Einstein', 'Heisenberg', 'Kelvin', 'Tesla', 'sum_households']

for col in cols:
    df_results.col=df_results.loc[~np.isnan(df_results[col]), col]
    df_r2 = pd.concat([df_r2, r2_score(df_results.groupby(df_results.loc['Timestamp'].dt.date)[col], df_results.groupby(df_results['Timestamp'].dt.date)[r2_base])], axis=1)

print(df_results[cols])
fig.add_traces([go.Scatter(x=df_r2.index, y=df_r2[col], name=col, opacity=0.7, mode='lines+markers') for col in cols], secondary_ys=[False] * len(cols))
fig.update_yaxes(title_text='Correlation value', range=[0, 1], secondary_y=False)

"""
cols_clouds = ['lowclouds', 'midclouds', 'highclouds', 'totalcloudcover']
fig.add_traces([go.Scatter(x=df_corr.index, y=df_corr[col], name=col, opacity=0.7, mode='lines+markers') for col in cols_clouds],
               secondary_ys=[True] * len(cols_clouds))
fig.update_yaxes(title_text='Cloud cover in %', range=[0, 100], secondary_y=True)

fig.update_layout(title_text='Daily Correlation to MeteoBlue Forecast vs. Cloud Cover (mean)', title_x=0.5,
                  xaxis_title='Date', template='plotly')
"""

fig.update_layout(title_text=f'Daily Correlation to MeteoBlue Forecast (resample: {resample_method})', title_x=0.5,
                  xaxis_title='Date', template='plotly')

fig.show(renderer='browser')

ValueError: Cannot index with multidimensional key